In [1]:
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np
from scipy import sparse 
import sys
from gaussiandata import SyntheticTarget
from onlinePrototypes import protonline
from onlineCriticism import crionline
#import greedy_functions
import math
import scipy

(1000, 2) => 100


In [2]:
SEED = 666
number_of_gaussians = 2
ex = SyntheticTarget(seed=SEED, dist_components=number_of_gaussians)

N = 1000
M = 10
data = ex.draw_sample(N)
print  data.size
print data[(2,3,4),:]
gamma = 2
kernel_matrix = rbf_kernel(data, gamma = gamma)

2000
[[ 2.58801945 -4.10201855]
 [-1.1745815  -3.81182871]
 [ 3.41529632 -3.59914177]]


In [3]:
def rbf(x,y, gamma):
    x = np.asarray(x)
    y = np.asarray(y)
    return scipy.exp(-gamma*np.linalg.norm(x-y))

def our_rbf_kernel(current_kernel_subset,  gamma, x_new, n, M, data, current_indices):
    print "we are in the update kernel"
    new_row = np.zeros(n+1)
    new_col = np.zeros(M)
    for i in range(n+1):
        new_row[i] = rbf(x = x_new, y = data[i], gamma = gamma)
         
    new_col = new_row[current_indices]
    new_row = np.matrix(new_row)
    new_col = np.matrix(new_col)

    new_kernel_subset = np.hstack((current_kernel_subset, new_col.T))
    new_kernel_subset = np.vstack((new_kernel_subset, new_row))
    
    return new_kernel_subset

In [36]:
gamma = 2
prototype_indices = np.arange(M) #assign first M points as prototypes. CONSIDER another method later.
criticism_indices = np.arange(M, 2*M) # assign second M points as prototypes.

#calculate the initial kernel matrix for the first M points
#current_kernel_matrix_proto = rbf_kernel(data[prototype_indices,:], gamma=gamma)


#current_kernel_matrix_criticism = rbf_kernel(data[criticism_indices,:], gamma = gamma) #to check with Petya

current_kernel_matrix = rbf_kernel(data[range(2*M)], gamma = gamma)
current_kernel_matrix_proto = current_kernel_matrix[:M]
current_kernel_matrix_criticism = current_kernel_matrix[range(M, 2*M)]


for i in range(2*M, N):
    
    new_point_index = i # assign the new data point index
    accept = 1 # decide whether to skip the criticism part or not 
    
    #prototype part
    #if accept == 1:
        #add new column and row to the kernel matrix for the new point; 
        #kernel matrix is always growing
        #current_kernel_subset,  gamma, x_new, n, timeseries):
        
    print "here"
    print "i"
    print i

    new_kernel_matrix_proto = our_rbf_kernel(current_kernel_subset = current_kernel_matrix_proto, gamma = gamma,  
                                                 x_new= data[i], n = i, M = M,  data = data[range(i+1)], 
                                                 current_indices = prototype_indices)

        #run the greedy algorithm on these M+1 data points to select M prototypes
        #print prototype_indices
    prototype_indices_temp = protonline(new_kernel_matrix_proto, 
                                                  selectedprotos = prototype_indices, m = M)
        #print "prototype_indices_temp"
        #print type(prototype_indices_temp)
        #print "prototype_indices"
        #print type(prototype_indices)
    prototype_indices_temp = np.sort(prototype_indices_temp)
    prototype_indices = np.sort(prototype_indices)
    print prototype_indices_temp
    print prototype_indices
    print all(prototype_indices_temp == prototype_indices)
    if all(prototype_indices_temp == prototype_indices):
        #accept = 0 
        print "the point is not accepted"
        new_kernel_matrix_criticism = our_rbf_kernel(current_kernel_subset = current_kernel_matrix_criticism, 
                                                     gamma = gamma,  x_new= data[i], n = i, M = M, 
                                                     data = data[range(i+1)], current_indices = criticism_indices)
        
        #run the greedy algorithm on these M+1 data points to select M criticism
        criticism_indices_temp = crionline(new_kernel_matrix_criticism, 
                                                  selectedprotos = prototype_indices, 
                                                  selectedcritic = criticism_indices, c = M, reg='logdet')

        
        if (criticism_indices_temp == criticism_indices).all():
            accept = 0
        
        else:
            criticism_indices = criticism_indices_temp # update the indices
            current_kernel_matrix_criticism = new_kernel_matrix_criticism # update the kernel matrix
        
    else:
        print "ciao"
        print "the point is accepted"
        print "prototype_indices"
        print prototype_indices
        print "prototype_indices_temp"
        print prototype_indices_temp
        print "current_kernel_matrix_proto"
        #print current_kernel_matrix_proto
        print current_kernel_matrix_proto.shape
        print "new_kernel_matrix_proto"
        #print new_kernel_matrix_proto
        print new_kernel_matrix_proto.shape
        print "difference"
        difference = np.setdiff1d(prototype_indices, prototype_indices_temp)
        print difference
        to_select = np.where(prototype_indices == difference)[0]
        print to_select
           
        prototype_indices = prototype_indices_temp # update the indices
        print new_kernel_matrix_proto.shape
        current_kernel_matrix_proto = np.delete(new_kernel_matrix_proto, to_select,0) # update the kernel matrix
        print current_kernel_matrix_proto.shape
  
    print i 
    print "accept"
    print accept 

here
i
20
we are in the update kernel
[ 0  1  2  3  4  6  7  8  9 20]
[0 1 2 3 4 5 6 7 8 9]
False
ciao
the point is accepted
prototype_indices
[0 1 2 3 4 5 6 7 8 9]
prototype_indices_temp
[ 0  1  2  3  4  6  7  8  9 20]
current_kernel_matrix_proto
(10, 20)
new_kernel_matrix_proto
(11, 21)
difference
[5]
[5]
(11, 21)
(10, 21)
20
accept
1
here
i
21
we are in the update kernel
[ 0  2  3  4  6  7  8  9 20 21]
[ 0  1  2  3  4  6  7  8  9 20]
False
ciao
the point is accepted
prototype_indices
[ 0  1  2  3  4  6  7  8  9 20]
prototype_indices_temp
[ 0  2  3  4  6  7  8  9 20 21]
current_kernel_matrix_proto
(10, 21)
new_kernel_matrix_proto
(11, 22)
difference
[1]
[1]
(11, 22)
(10, 22)
21
accept
1
here
i
22
we are in the update kernel
[ 0  2  3  4  6  7  8  9 20 22]
[ 0  2  3  4  6  7  8  9 20 21]
False
ciao
the point is accepted
prototype_indices
[ 0  2  3  4  6  7  8  9 20 21]
prototype_indices_temp
[ 0  2  3  4  6  7  8  9 20 22]
current_kernel_matrix_proto
(10, 22)
new_kernel_matrix_proto
(

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [34]:
a = np.array([5,2,1,6])
b = np.array([6,2,5,1])
diff =  np.setdiff1d(a,b)
print diff

[]


In [24]:
to_select = np.where(a == diff)[0]
print to_select
print to_select-1

[2]
[1]


In [21]:
a[to_select]

array([3])

In [26]:
np.delete(a, to_select,0)

array([1, 2, 5])

In [35]:

all(a == b)

False